# Multi-level OLS modeling

Goals:
 - Working with panel data
 - More experience with multivariate regression
 - Simple example of multilevel modeling

In [ ]:
df <- read.csv("coffee.csv")

In [ ]:
head(df)

In [ ]:
length(unique(df$region))

In [ ]:
library(ggplot2)
ggplot(subset(df, region %in% unique(df$region)[1:16]), aes(year, produced)) +
  facet_wrap(~ region, scales='free_y') + geom_line()

In [ ]:
ggplot(subset(df, region %in% unique(df$region)[1:16]), aes(year, tavg)) +
  facet_wrap(~ region) + geom_line()

In [ ]:
plot(density(df$tavg))

In [ ]:
summary(lm(yield ~ tavg, data=df))

In [ ]:
df$tavg2 = df$tavg^2

In [ ]:
summary(lm(yield ~ tavg + tavg2, data=df))

In [ ]:
projdf = data.frame(tavg=seq(24, 26, length.out=100))

In [ ]:
projdf$tavg2 = projdf$tavg^2

In [ ]:
mod = lm(yield ~ tavg + tavg2, data=df)

In [ ]:
projdf$yield = predict(mod, projdf)

In [ ]:
ggplot(projdf, aes(tavg, yield)) + geom_line()

In [ ]:
?predict

In [ ]:
?predict.lm

In [ ]:
projdf2 = cbind(projdf, predict(mod, projdf, interval='confidence'))

In [ ]:
head(projdf2)

In [ ]:
ggplot(projdf2, aes(tavg, fit)) + geom_line() + geom_ribbon(aes(ymin=lwr, ymax=upr), alpha=.5)

In [ ]:
library(lfe)

In [ ]:
summary(felm(yield ~ tavg + tavg2 | region, data=df))

In [ ]:
mod2 = felm(yield ~ tavg + tavg2 | region, data=df)

In [ ]:
projdf3 = cbind(projdf, predict(mod2, projdf))

In [ ]:
predict.felm <- function(object, newdata, se.fit = FALSE,
                         interval = "none",
                         level = 0.95, special.vcov=NULL){
  if(missing(newdata)){
    stop("predict.felm requires newdata and predicts for all group effects = 0.")
  }

  tt <- terms(object)
  Terms <- delete.response(tt)
  attr(Terms, "intercept") <- 0

  m.mat <- model.matrix(Terms, data = newdata)
  m.coef <- as.numeric(object$coef)
  fit <- as.vector(m.mat %*% object$coef)
  fit <- data.frame(fit = fit)

  if(se.fit | interval != "none"){
      if (!is.null(special.vcov)) {
          vcov_mat <- special.vcov
      } else if (!is.null(object$clustervcv)){
          vcov_mat <- object$clustervcv
      } else if (!is.null(object$robustvcv)) {
          vcov_mat <- object$robustvcv
      } else if (!is.null(object$vcv)){
          vcov_mat <- object$vcv
      } else {
          stop("No vcv attached to felm object.")
      }
      se.fit_mat <- sqrt(diag(m.mat %*% vcov_mat %*% t(m.mat)))
  }
  if(interval == "confidence"){
    t_val <- qt((1 - level) / 2 + level, df = object$df.residual)
    fit$lwr <- fit$fit - t_val * se.fit_mat
    fit$upr <- fit$fit + t_val * se.fit_mat
  } else if (interval == "prediction"){
    stop("interval = \"prediction\" not yet implemented")
  }
  if(se.fit){
    return(list(fit=fit, se.fit=se.fit_mat))
  } else {
    return(fit)
  }
}

In [ ]:
projdf3 = cbind(projdf, predict.felm(mod2, projdf, interval="confidence"))

In [ ]:
head(projdf3)

In [ ]:
ggplot(projdf2, aes(tavg, fit)) + geom_line(aes(colour='LM')) + geom_ribbon(aes(ymin=lwr, ymax=upr, colour='LM'), alpha=.5) +
  geom_line(data=projdf3, aes(colour='FELM')) + geom_ribbon(data=projdf3, aes(ymin=lwr, ymax=upr, colour='FELM'), alpha=.5)

In [ ]:
projdf.diff = data.frame(tavg=projdf$tavg - 25, tavg2=projdf$tavg2 - 25^2)

In [ ]:
projdf3 = cbind(projdf, predict.felm(mod2, projdf.diff, interval="confidence"))

In [ ]:
ggplot(projdf2, aes(tavg, fit)) + geom_line(aes(colour='LM')) + geom_ribbon(aes(ymin=lwr, ymax=upr, colour='LM'), alpha=.5) +
  geom_line(data=projdf3, aes(colour='FELM')) + geom_ribbon(data=projdf3, aes(ymin=lwr, ymax=upr, colour='FELM'), alpha=.5)

## Want elevation by state

Get ADM2 shapefile from https://gadm.org/
Get GLOBE digital elevation model from https://www.ngdc.noaa.gov/mgg/topo/globe.html

Use "Spatial statistics" in QGIS to get average elevation by ADM2 region.

In [ ]:
library(PBSmapping)
polydata = attr(importShapefile("brazil-adm2.shp"), "PolyData")

In [ ]:
head(polydata)

In [ ]:
states = read.csv("brazil-states.csv")

In [ ]:
head(states)

In [ ]:
library(dplyr)

In [ ]:
polydata2 = polydata %>% left_join(states, by=c('NAME_1'='Name'))

In [ ]:
nrow(polydata)

In [ ]:
nrow(polydata2)

In [ ]:
polydata2$mergename = paste0(polydata2$NAME_2, " (", polydata2$Code, ")")

In [ ]:
head(polydata2)

In [ ]:
df2 = df %>% left_join(polydata2, by=c("region"="mergename"))

In [ ]:
nrow(df)

In [ ]:
nrow(df2)

In [ ]:
head(df2)

In [ ]:
plot(df2$tavg, df2$elev_mean)

In [ ]:
df2$elev.ishi = df2$elev_mean > 500

In [ ]:
summary(lm(yield ~ tavg * elev_mean + tavg2 * elev_mean, data=df2))

In [ ]:
quantile(df2$elev_mean, na.rm=T)

In [ ]:
mod = lm(yield ~ tavg * elev_mean + tavg2 * elev_mean, data=df2)
projdf4.hi = predict(mod, cbind(projdf, elev_mean=1000), interval="confidence")
projdf4.lo = predict(mod, cbind(projdf, elev_mean=400), interval="confidence")

In [ ]:
projdf4 = rbind(cbind(elev='1000 m', projdf, projdf4.hi), cbind(elev='400 m', projdf, projdf4.lo))

In [ ]:
head(projdf4)

In [ ]:
ggplot(projdf4, aes(tavg, fit, colour=elev)) + geom_line() + geom_ribbon(aes(ymin=lwr, ymax=upr), alpha=.5)

In [ ]:
summary(felm(yield ~ tavg * elev_mean + tavg2 * elev_mean | region, data=df2))

In [ ]:
summary(felm(yield ~ tavg + tavg2 + tavg : elev_mean + tavg2 : elev_mean | region, data=df2))

In [ ]:
mod = felm(yield ~ tavg + tavg2 + tavg : elev_mean + tavg2 : elev_mean | region, data=df2)
projdf5.hi = predict.felm(mod, cbind(projdf.diff, elev_mean=1000), interval="confidence")
projdf5.lo = predict.felm(mod, cbind(projdf.diff, elev_mean=400), interval="confidence")

In [ ]:
projdf5 = rbind(cbind(elev='1000 m', projdf, projdf5.hi), cbind(elev='400 m', projdf, projdf5.lo))

In [ ]:
head(projdf5)

In [ ]:
ggplot(projdf5, aes(tavg, fit, colour=elev)) + geom_line() + geom_ribbon(aes(ymin=lwr, ymax=upr), alpha=.5)